- dgenerator
- date: 2020-08-06
- maintainer: YZK

In [184]:
# %%bash
# jupyter nbconvert --to script dgenerator.ipynb

[NbConvertApp] Converting notebook dgenerator.ipynb to script
[NbConvertApp] Writing 21562 bytes to dgenerator.py


In [6]:
import logging
import os
import re
import sys
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
from sklearn import preprocessing

try:
    thisd = os.path.dirname(os.path.realpath(__file__))
    sys.path.append(thisd)
    from mdfloader import Dataset
    from parallel import runFunctionsInParallel
    from textloader import textloader 
except Exception as E:
    logging.warning(E)
    if __name__ == "__main__":
        from mdfloader import Dataset
        from parallel import runFunctionsInParallel
        from textloader import textloader    
    else:
        from lib.mdfloader import Dataset
        from lib.parallel import runFunctionsInParallel
        from lib.textloader import textloader

floader   = Dataset.floader
getGI     = Dataset.getGI
qcfparser = Dataset.qcfparser
getids    = textloader.get_id

In [3]:
class dgenerator():
    def __init__(self, resd=None, ind=None, cmpltd=None, npyd=None, parsed=None, 
                 gif="stations.txt",
                 cwbfmt=".QPESUMS_STATION.10M.mdf",
                 autofmt=".QPESUMS_STATION.15M.mdf",
                 pfmt=".QPESUMS_STATION.txt"):
        
        self.ind     = ind
        self.cmpltd  = cmpltd
        self.npyd    = npyd
        self.parsed  = parsed
        self.cwbfmt  = cwbfmt
        self.autofmt = autofmt
        self.pfmt    = pfmt
        self.vnames  = ["Temp", "RH", "Pres", "Precp"]
        self.stnids  = None
        self.nstn    = None
        
        if resd is not None:
            try:
                self.raw_gi, self.stnids, self.nstn = getGI("{0}/{1}".format(resd, gif))
            except Exception as E:
                logging.warning("dgenerator-23: {}".format(E))
                self.stnids = getids("{0}/{1}".format(resd, gif))
                self.nstn   = len(self.stnids)
        else:
            try:
                self.stnids = getids("{0}".format(gif))
                self.nstn   = len(self.stnids)
            except Exception as E:
                logging.warning("dgenerator-31: {}".format(E))
                self.raw_gi, self.stnids, self.nstn = getGI("{0}".format(gif))

        self.vrange = {"Temp": [-20.0, 50.0],
                       "RH": [0.0, 100.0], 
                       "Pres": [600.0, 1100.0], 
                       "Precp": [0.0, 220.0]}
        
        self.vname2abbr = {"Temp": "TT", 
                           "RH": "RH", 
                           "Pres": "PP", 
                           "Precp": "RR"}
    @property
    def stnids(self):    
        return self._stnids
        
    @stnids.setter
    def stnids(self, val):
        self._stnids = val
        
    @property
    def vrange(self):
        return self._vrange
    
    @vrange.setter
    def vrange(self, val):
        self._vrange = val
    
    @vrange.deleter
    def vrange(self):
        del self._vrange
        print("dgenerator-35: self._vrange has been deleted.")
        
    @staticmethod
    def series_to_supervised(data, n_in=1, n_out=1, dropnan=True, vnames=None):

        '''
            convert series to supervised learning
            create a dataset with columns t - n_in, ..., t - 2, t - 1, t, t + 1, ..., t + (n_out - 1),
            if n_in = 2 and n_out = 2, then get a dataset with columns t - 2, t - 1, t, t + 1
        '''

        n_vars = 1 if type(data) is list else data.shape[1]

        if isinstance(data, pd.DataFrame):
            df = data
        else:
            df = pd.DataFrame(data)

        cols, names = list(), list()

        # input sequence (t-n_in, ..., t-1)
        for i in range(n_in, 0, -1):
            cols.append(df.shift(i))
            if vnames is not None and len(vnames) == n_vars:
                names += ["{0}(t-{1})".format(vname, i) for vname in vnames]
            else:
                names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]

        # forecast sequence (t, t+1, ..., t+n_out-1)
        for i in range(0, n_out):
            cols.append(df.shift(-i))
            if i == 0:
                if vnames is not None and len(vnames) == n_vars:
                    names += ["{0}(t)".format(vname) for vname in vnames]
                else:
                    names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
            else:
                if vnames is not None and len(vnames) == n_vars:
                    names += [("{0}(t+{1})".format(vname, i)) for vname in vnames]
                else:
                    names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]

        # put it all together
        agg = pd.concat(cols, axis=1)
        agg.columns = names
        # drop rows with NaN values
        if dropnan:
            agg.dropna(inplace=True)
        return agg
            
    @staticmethod
    def mdfinspector(ind, cwbfmt, autofmt, tperiod=None, qc_log=None, cmpltd=None, rtqc=False):

        '''
            check what time do mdf exist 
            tperiod: time period for inspecting mdf, a list like [stime, etime]
            cwbfmt: suffix of filename
            autofmt: suffix of filename
        '''
        
        if cmpltd is None:
            cmpltd = ind
        
        if tperiod is not None:
            if len(tperiod) == 1:
                qc_stime = datetime.strptime(str(tperiod[0]), "%Y%m%d%H%M")
                qc_etime = datetime.strptime(str(tperiod[0]), "%Y%m%d%H%M")
                tperiod.append(tperiod[0])
            elif len(tperiod) == 2:
                qc_stime = datetime.strptime(str(tperiod[0]), "%Y%m%d%H%M")
                qc_etime = datetime.strptime(str(tperiod[1]), "%Y%m%d%H%M")
            else:
                tperiod = None
                print("Warning-mdfinspector-178: time period is a list with length 2.")
                
        if tperiod is None:
            qc_stime = datetime.now() - timedelta(years=365)
            qc_etime = datetime.now()

        fnames = os.listdir(ind)
        fnames.sort()

        cwbfre = re.compile(pattern=r"^([0-9]{10})([0-9]{2})" + cwbfmt)
        autofre = re.compile(pattern=r"^([0-9]{10})([0-9]{2})" + autofmt)

        qcdtimes = [] # a integer list

        dtnow = datetime.now()

        # check if cwb or auto exists (one of them)
        nf = 0
        for idx, tmp in enumerate(fnames):

            autofmatch = autofre.match(tmp)
            cwbfmatch = cwbfre.match(tmp)
            matched = False
            dtobj = None

            if autofmatch is not None:
                YYYYmmddHHMM = autofmatch.group(1) + autofmatch.group(2)
                if int(YYYYmmddHHMM) not in qcdtimes:
                    if os.path.isfile("{}/{}{}".format(ind, YYYYmmddHHMM, cwbfmt)) or os.path.isfile("{}/{}{}".format(cmpltd, YYYYmmddHHMM, cwbfmt)):  # check cwb
                        dtobj = datetime.strptime(YYYYmmddHHMM, "%Y%m%d%H%M")
            elif cwbfmatch is not None:
                YYYYmmddHHMM = cwbfmatch.group(1) + cwbfmatch.group(2)
                if int(YYYYmmddHHMM) not in qcdtimes:
                    if os.path.isfile("{}/{}{}".format(ind, YYYYmmddHHMM, autofmt)) or os.path.isfile("{}/{}{}".format(cmpltd, YYYYmmddHHMM, autofmt)):  # check cwb
                        dtobj = datetime.strptime(YYYYmmddHHMM, "%Y%m%d%H%M")

            if dtobj is not None:
                if rtqc:
                    if abs((dtnow - dtobj).total_seconds() / (60 * 60)) > (8 + 24):  # obs file is UTC time
                        continue
                    matched = True

                if tperiod is not None:                                              # check qc time period
                    if not (qc_stime <= dtobj and dtobj <= qc_etime):
                        continue
                    matched = True

                if matched:
                    qcdtimes.append(int(YYYYmmddHHMM))
                    if qc_log is not None:
                        nf += 1
                        qc_log.write("  {0:3d}. {1}\n".format(nf, YYYYmmddHHMM + cwbfmt))
                        nf += 1
                        qc_log.write("  {0:3d}. {1}\n".format(nf, YYYYmmddHHMM + autofmt))

        return sorted(set(qcdtimes))
    
    
    def hrfgenerator(self, tperiod, n_in=6, n_out=1, batchsize=24, mode="train", fnpy=True, rescale=True, reformat=True, vstack=True, dropnan=True, generator=False):
        
        ind        = self.ind
        npyd       = self.npyd
        stnids     = self.stnids
        nstn       = self.nstn
        vnames     = self.vnames
        vrange     = pd.DataFrame(self.vrange)
        vnames_    = vrange.columns.tolist()
        vname2abbr = self.vname2abbr

        assert vnames == vnames_
        
        logging.info("hrfgenerator-vnames-199: {}".format(vnames))

        if tperiod is not None:
            if len(tperiod) == 1:
                sdtime = tperiod[0]
                edtime = tperiod[0]
                tperiod.append(tperiod[0])
            elif len(tperiod) == 2:
                sdtime = tperiod[0]
                edtime = tperiod[1]
            else:
                tperiod = None
                logging.error("hrfgenerator-209: tperiod is None")
                sys.exit(-1)    

        if fnpy:
            if npyd is None:  
                logging.error("hrfgenerator-214: npyd is None")
                sys.exit(-1)
                
            darray   = np.load("{}/hrf_{}_{}_{}.npy".format(npyd, tperiod[0], tperiod[1], mode))
            qcdtimes = np.load("{}/qcdtime_{}_{}_{}.npy".format(npyd, tperiod[0], tperiod[1], mode)).tolist()
            stnids_  = np.load("{}/stnid_{}_{}_{}.npy".format(npyd, tperiod[0], tperiod[1], mode)).tolist()
            nsize    = len(qcdtimes)
            nstn_    = len(stnids_)
            
            if stnids is not None:
                assert stnids == stnids_
            else:
                stnids = stnids_
            
            if nstn is not None:
                assert nstn == nstn_
            else:
                nstn = nstn_
            
        else:
            if stnids is None or nstn is None:
                logging.error("hrfgenerator-220: stnids or nstn is None.")
                sys.exit(-1)
            
            obs = []

            for vname_ in vnames:
                vname = vname2abbr[vname_]
                ind_ = "{0}/{1}".format(ind, vname)
                obs_, stnid, YYYYmmddHH_, quantity = textloader.hrf(ind_, stnids, sdtime, edtime, hrfp="hr_{}".format(vname.lower()))
                logging.info("hrfgenerator-244: quantity of {}: {}".format(vname, quantity))
                YYYYmmddHH = YYYYmmddHH_
                obs.append(obs_.T)
            
            darray   = np.stack(obs, axis=2)
            nsize    = darray.shape[0]
            nstn     = darray.shape[1]
            qcdtimes = YYYYmmddHH
            stnids   = stnid
            
            if npyd is not None:
                np.save("{}/hrf_{}_{}_{}.npy".format(npyd, tperiod[0], tperiod[1], mode), arr=darray)
                np.save("{}/qcdtime_{}_{}_{}.npy".format(npyd, tperiod[0], tperiod[1], mode), arr=np.array(qcdtimes))
                np.save("{}/stnid_{}_{}_{}.npy".format(npyd, tperiod[0], tperiod[1], mode), arr=np.array(stnids))

        return dgenerator._dprocessing(darray, qcdtimes, stnids, vrange, n_in=n_in, n_out=n_out, batchsize=batchsize, rescale=rescale, reformat=reformat, vstack=vstack, dropnan=dropnan, generator=generator)

    
    def mdfgenerator(self, tperiod, n_in=6, n_out=1, batchsize=24, mode="train", fnpy=True, rescale=True, reformat=True, vstack=True, dropnan=True, generator=False):

        '''
            {mode} is just suffix of npy files, ex. mdf_{mode}.npy, qcdtime_{mode}.npy, stnid_{mode}.npy
            if {fnpy} is True, then load data from npy files
            if {generator} is True, then {batchesize} will be activated and get a iterator (generator)
            if {rescale} is True, then data will be normalized
            if {reformat} is True, then data array will be reformatted by staticmethod "series_to_supervised" with arguments {n_in}, {n_out}
            if {vstack} is True, then observations of all stations (dim = (nsize, :)) will stack vertically (dim = (nsize * nstn, :))
            if {dropnan} is True, then data less than minimum in self.vrange will be set to np.nan and dropped if {vstack} is True
        '''
        
        ind     = self.ind
        cmpltd  = self.cmpltd
        npyd    = self.npyd
        parsed  = self.parsed
        cwbfmt  = self.cwbfmt 
        autofmt = self.autofmt 
        pfmt    = self.pfmt
        stnids  = self.stnids
        nstn    = self.nstn
        vnames  = self.vnames
        vrange  = pd.DataFrame(self.vrange)
        vnames_ = vrange.columns.tolist()
        
        assert vnames == vnames_
        
        logging.info("mdfgenerator-vnames-290: {}".format(vnames))
        
        if tperiod is not None:
            if len(tperiod) == 1:
                sdtime = datetime.strptime(str(tperiod[0]), "%Y%m%d%H%M")
                edtime = datetime.strptime(str(tperiod[0]), "%Y%m%d%H%M")
                tperiod.append(tperiod[0])
            elif len(tperiod) == 2:
                sdtime = datetime.strptime(str(tperiod[0]), "%Y%m%d%H%M")
                edtime = datetime.strptime(str(tperiod[1]), "%Y%m%d%H%M")
            else:
                tperiod = None
                logging.error("mdfgenerator-303: tperiod is None.")
                sys.exit(-1)

        if fnpy:  # data from npy
            
            if npyd is None:  
                logging.error("mdfgenerator-309: npyd is None")
                sys.exit(-1)
                
            darray   = np.load("{}/mdf_{}_{}_{}.npy".format(npyd, tperiod[0], tperiod[1], mode))
            qcdtimes = np.load("{}/qcdtime_{}_{}_{}.npy".format(npyd, tperiod[0], tperiod[1], mode)).tolist()
            stnids_  = np.load("{}/stnid_{}_{}_{}.npy".format(npyd, tperiod[0], tperiod[1], mode)).tolist()
            
            nsize = len(qcdtimes)
            nstn_ = len(stnids_)  # it's a list
            
            if stnids is not None:
                assert stnids == stnids_
            else:
                stnids = stnids_
                
            if nstn is not None:
                assert nstn == nstn_
            else:
                nstn = nstn_
                
        else:    
            if stnids is None or nstn is None:
                logging.error("mdfgenerator-332: stnids or nstn is None.")
                sys.exit(-1)

            qcdtimes = dgenerator.mdfinspector(ind, cwbfmt, autofmt, tperiod=tperiod)
            nsize = len(qcdtimes)
            
            darray = np.ndarray(shape=(nsize, nstn, 4))  # Temp, RH, Pres, Precp
            darray.fill(-999)
        
            dtime_ = sdtime
            counts = 0
            jobs = []
            jnames = []

            while dtime_ <= edtime:
                dtime = dtime_.strftime("%Y%m%d%H%M")
                counts += 1
                jnames.append("{0:>04d}-{1}".format(counts, dtime))
                jobs.append([qcfparser, [dtime, [ind, cmpltd, parsed]], {"outdir": parsed}])  # don't output if outdir is None
                if dtime_ == edtime:
                    ret = runFunctionsInParallel(jobs, names=jnames, 
                                                 offsetsSeconds=0, maxAtOnce=None, 
                                                 parallel=True, allowJobFailure=True)
                    for qidx in range(counts):
                        id_     = ret[1][qidx][0]
                        array_  = ret[1][qidx][1]
                        chname_ = ret[1][qidx][2]
                        dt_     = ret[1][qidx][3]
                        ttuple_  = datetime.strptime(dt_, "%Y%m%d%H%M").timetuple()
                        if array_ is None:  # to take obs from parsed files in order to make a time series is continuous
                            pdarray = floader(parsed, "{}{}".format(dt_, pfmt))
                            if pdarray is None:
                                logging.warning("mdfgenerator-363: why array_ is None on {}?".format(dt_))
                                continue
                            id_     = pdarray[0:, 0]
                            array_  = pdarray[0:, 1:-1]
                            chname_ = pdarray[0:, -1]

                        for wrow, mdfid in enumerate(id_):

                            if not (mdfid in stnids):
                               continue

                            id_idx = stnids.index(mdfid)
                            dt_idx = qcdtimes.index(int(dt_))

                            darray[dt_idx, id_idx, 0] = float(array_[wrow, 3])  # Temp
                            darray[dt_idx, id_idx, 1] = float(array_[wrow, 4])  # RH
                            darray[dt_idx, id_idx, 2] = float(array_[wrow, 5])  # Pres
                            darray[dt_idx, id_idx, 3] = float(array_[wrow, 6])  # Precp

                    counts = 0
                    jobs = []
                    jnames = []

                dtime_ = dtime_ + timedelta(minutes=10)
                
            if npyd is not None:
                np.save("{}/mdf_{}_{}_{}.npy".format(npyd, tperiod[0], tperiod[1], mode), arr=darray)
                np.save("{}/qcdtime_{}_{}_{}.npy".format(npyd, tperiod[0], tperiod[1], mode), arr=np.array(qcdtimes))
                np.save("{}/stnid_{}_{}_{}.npy".format(npyd, tperiod[0], tperiod[1], mode), arr=np.array(stnids))        
                
        return dgenerator._dprocessing(darray, qcdtimes, stnids, vrange, n_in=n_in, n_out=n_out, batchsize=batchsize, rescale=rescale, reformat=reformat, vstack=vstack, dropnan=dropnan, generator=generator)
            
    @staticmethod            
    def _dprocessing(darray, qcdtimes, stnids, vrange, n_in=6, n_out=1, batchsize=24, rescale=True, reformat=True, vstack=True, dropnan=True, generator=False):
        
        '''
            darray.shape = (nsize, nstn, 4)
        '''
        
        vnames = vrange.columns.to_list()
        nsize  = len(qcdtimes)
        nstn   = len(stnids)
        
        if dropnan:
            for idx, vname in enumerate(vnames):
                logging.info("_dprocessing-408: drop missing values of {}".format(vname))
                darray[:, :, idx][np.where(darray[:, :, idx] < vrange[vname][0])] = np.nan           
                
        ts2supervise = np.copy(darray)  # dim = [nsize, nstn, 4]
        
        # darray[:, :, idx]
        #    idx       |    0 |  1 |    2 |     3
        # ------------------------------------------
        #    variable  | Temp | RH | Pres | Precp
        rescaled = []
        if rescale:
            scaler = preprocessing.MinMaxScaler()
            scaler.fit(vrange.values)
            for id_idx, id_ in enumerate(stnids):
                rescaled.append(scaler.transform(darray[:, id_idx, :]))
                
            ts2supervise = np.stack(rescaled, axis=1)  # dim = [nsize, nstn, 4]
        else:    
            for id_idx, id_ in enumerate(stnids):
                rescaled.append(darray[:, id_idx, :])
        
        reformated = []
        if reformat:
            for id_idx, id_ in enumerate(stnids):
                reformated.append(dgenerator.series_to_supervised(rescaled[id_idx], n_in=n_in, n_out=n_out, dropnan=False, vnames=vnames).values)
            
            if vstack:
                ts2supervise = np.vstack(reformated)  # dim = [nsize * nstn, (n_in + n_out) * 4]
                if dropnan:
                    ts2supervise = ts2supervise[~np.isnan(ts2supervise).any(axis=1)]
                nsize_ = ts2supervise.shape[0]
            else:
                ts2supervise = np.stack(reformated, axis=1)  # dim = [nsize, nstn, (n_in + n_out) * 4]
                nsize_ = nsize
        
        def _diterator():
            # variables can still identify in this function scope
            while True:
                idx = np.random.choice(nsize_, batchsize, replace=False)
                if reformat:
                    if vstack:
                        yield ts2supervise[idx, :(-4)*n_out], ts2supervise[idx, (-4)*n_out:]
                    else:
                        yield ts2supervise[idx, :, :(-4)*n_out], ts2supervise[idx, :, (-4)*n_out:]

        if not generator:
            return [ts2supervise, qcdtimes, stnids, darray]
        else:
            logging.info("_dprocessing-456: data generator has been built.")
            return _diterator()
        

In [4]:
if __name__ == "__main__":
    
    logging.getLogger().setLevel(logging.DEBUG)
    
    resd    = "/NAS-DS1515P/users1/T1/res"
    ind     = "/NAS-DS1515P/users1/realtimeQC/ftpdata"
    cmpltd  = ind
    npyd    = "/home/yuzhe/DataScience/dataset"
    parsed  = "/home/yuzhe/DataScience/dataset/parsed"
    tperiod = [202005200000, 202005202350]
    cwbfmt  = ".QPESUMS_STATION.10M.mdf"
    autofmt = ".QPESUMS_STATION.15M.mdf"
    pfmt    = ".QPESUMS_STATION.txt"
    
    dg = dgenerator(resd, ind=ind, npyd=npyd, cwbfmt=cwbfmt, autofmt=autofmt, pfmt=pfmt)
#     mdfs = dg.mdfgenerator(tperiod, n_in=6, n_out=1, mode="train", rescale=True, reformat=True, vstack=True, fnpy=False, generator=False)            
    print(dg.vrange)
    
    dg1 = dgenerator(gif="{}/stations.txt".format(resd), ind=ind, npyd=npyd, cwbfmt=cwbfmt, autofmt=autofmt, pfmt=pfmt)
#     mdfs = dg1.mdfgenerator(tperiod, n_in=6, n_out=1, mode="train", rescale=True, reformat=True, vstack=True, fnpy=False, generator=False)            

    ind = "/NAS-129/users1/T1/DATA/YY/ORG/HR1"
    npyd = "/home/yuzhe/DataScience/dataset"
    tperiod = [2012123101, 2012123124]
#     stninfo = "/home/yuzhe/CODE/ProgramT1/GRDTools/SRC/RES/GI/RR_analysis_grid_stationlist.txt"
    stninfo = "/home/yuzhe/CODE/ProgramT1/GRDTools/SRC/RES/GI/1500_decode_stationlist_without_space.txt"

    dg2 = dgenerator(ind=ind, gif=stninfo, npyd=npyd)
    hrfs = dg2.hrfgenerator(tperiod, n_in=6, n_out=1, mode="train", rescale=True, reformat=True, vstack=True, fnpy=False, generator=False)
    
    print(dg2.stnids)

INFO:root:hrfgenerator-vnames-199: ['Temp', 'RH', 'Pres', 'Precp']
INFO:root:textloader-hrf-138: sdtime: 2012123101, edtime: 2012123124, nsample: 24


{'Temp': [-20.0, 50.0], 'RH': [0.0, 100.0], 'Pres': [600.0, 1100.0], 'Precp': [0.0, 220.0]}


INFO:root:textloader-hrf-quantity: 0.2851904761904762
INFO:root:hrfgenerator-244: quantity of TT: 0.2851904761904762
INFO:root:textloader-hrf-138: sdtime: 2012123101, edtime: 2012123124, nsample: 24
INFO:root:textloader-hrf-quantity: 0.199
INFO:root:hrfgenerator-244: quantity of RH: 0.199
INFO:root:textloader-hrf-138: sdtime: 2012123101, edtime: 2012123124, nsample: 24
INFO:root:textloader-hrf-quantity: 0.26242857142857146
INFO:root:hrfgenerator-244: quantity of PP: 0.26242857142857146
INFO:root:textloader-hrf-138: sdtime: 2012123101, edtime: 2012123124, nsample: 24
INFO:root:textloader-hrf-quantity: 0.548
INFO:root:hrfgenerator-244: quantity of RR: 0.548
INFO:root:_dprocessing-408: drop missing values of Temp
INFO:root:_dprocessing-408: drop missing values of RH
INFO:root:_dprocessing-408: drop missing values of Pres
INFO:root:_dprocessing-408: drop missing values of Precp


['466900', '466920', '467490', '467410', '466940', '466990', '467660', '467590', '466950', '467350', '467440', '467080', '467610', '467540', '466910', '466930', '467650', '467530', '467550', '467620', '467480', '467770', '467300', '467060', '467571', '467420', '466880', '467780', '467990', '467110', '467050', '466850', '467790', '467570', '466921', '467411', 'A0A9MA', 'A0C54A', 'D2F23A', 'A0Z08A', 'A0T78A', 'A0G72A', 'A0K42A', 'A0W08A', 'A0W03A', '46810A', '72C440', '82C160', '72D080', 'K2E360', 'G2F820', '72G600', 'U2H480', '72K220', '72M360', 'B2N890', '72Q010', 'B2Q810', '72U480', '72T250', '72S200', '72S590', '82A750', '82H840', '466860', '466960', '467400', '467380', '467900', '467360', '467880', '467890', '467860', '467870', '46745R', '46750R', '46746R', '46770R', '46743R', '46756R', '46734R', '46760R', '46758R', '46763R', 'C0A520', 'C0A530', 'C0A540', 'C0A550', 'C0A560', 'C0A570', 'C0A580', 'C0A640', 'C0A650', 'C0A660', 'C0A680', 'C0A710', 'C0A860', 'C0A870', 'C0A880', 'C0A890',